In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
# utils

def _cal_d(X, m, E, r):
    
    """
    # at (step = k), we have the best feature x_k, alpha_k, and r_k
    # then calculate the moving direction for the next step, d_k such that:
    
        # <X*E_k, X*d_k> = constant_1*S_k,
        
    # here S_k = (sign(<x_0, r_k>), ..., sign(<x_k, r_k>)), a column vector
    # and, constant_1 is any constant that != 0
    
    # According to (step = k-1), we have:
    
        # <X*E_k, r_k> = constant_2*S_k,
    
    # then we got:
    
        # <X*E_k, X*d_k> = <X*E_k, r_k>
    """
        
    if E.shape == (m, ):
        E = E[:, np.newaxis]
        
    Z = X.dot(E)
    G = Z.T.dot(Z)
    G_inv = np.linalg.inv(G)
    _d = E.dot(G_inv)
    _d_ = _d.dot(Z.T)
    d = _d_.dot(r)
    return d

def _cal_alpha(X, x, pre_best_x, pre_r, d):
    """
    # alpha need to meet:
    
    #    C0: |<x_k, r_k>| = |<x_k-1, r_k>|
    
    # here, r_k = r_k-1 - alpha*X*d_k-1
    
    # with constraint:
    
    #    C1: |<x_k, r_k>| < |<x_k-1, r_k-1|
    
    
    # according to:
    #    L1: |<x_k, r_k>| = |<x_k-1, r_k>|
    # and,
    #    L2: <x_k, alpha*X.dot(d)> = <x_k-1, r_k-1>
    # and,
    #    L3: r_k = r_k-1 - alpha*X*d_k-1
    
    # constraint C1 equals to:
    
    #    |<x_k-1, r_k>| = |<x_k-1, r_k-1 - alpha*X*d_k-1>| = 
    #        |<x_k-1, r_k-1> - alpha*<x_k-1, X*d_k-1>| = 
    #        |<x_k-1, r_k-1> - alpha*<x_k-1, r_k-1>| < |<x_k-1, r_k-1|

    # that constraint C1 requires:
    
    # |1-alpha| < 1 
    """
    
    alpha = None
    
    X_d = X.dot(d)
    
    x_pre_r = x.dot(pre_r)
    pre_best_x_pre_r = pre_best_x.dot(pre_r)
    
    x_ = x.dot(X_d)
    pre_best_x_ = pre_best_x.dot(X_d)
    
    alpha_plus = (x_pre_r - pre_best_x_pre_r)/(x_ - pre_best_x_)
    alpha_minus = (x_pre_r + pre_best_x_pre_r)/(x_ + pre_best_x_)
    
    if 0 < alpha_plus < 2:
        alpha = alpha_plus
    if (0 < alpha_minus < 2) and (alpha_minus < alpha_plus):
        alpha = alpha_minus
    
    return alpha

def _cal_beta(alpha_best, beta, pre_d):
    beta = beta + alpha_best*pre_d
    return beta

def _last_alpha(X, r, x, d):
    """
    # <last_x, last_r> = 0
    #last_r = r - alpha*X*d
    """
    a = r.dot(x)
    delta = X.dot(d)
    b = delta.dot(x)
    return a/b

In [3]:
# Least Angle Regression

def lars(X_raw, y_raw):
    """
    References: http://statweb.stanford.edu/~tibs/ftp/lars.pdf
    
    """
    
    # Important: assume all the feature are linearly independent
    
    
    
    # 1. init
    standScaler = StandardScaler()
    X = standScaler.fit_transform(X_raw, y_raw)
    y = y_raw
    m = X.shape[1]
    
    beta_path = np.zeros(m)
    
    # F: all feature indices; A: chosen feature indices; L: the left feature indices
    F = np.arange(m)
    A = []
    L = list(set(F) - set(A))
    
    # X.dot(E) = chosen features ordered as chosen order
    E = None
    
    beta = np.zeros(m)
    y_hat = X.dot(beta)
    r = y - y_hat
    
    # 2. step 0
    
    # 2.1 step 0 results for the first best feature
    best_index = np.argmax(np.abs(np.matmul(X.T, r)))
    best_x = X[:, best_index]
    
    # 2.2 prepare for the next step
    A.append(best_index)
    L = list(set(F) - set(A))
    
    e = np.zeros(m)
    np.put(e, best_index, 1)
    E = e
    
    d = _cal_d(X, m, E, r)
    
    # 3. start iteration
    while L != []:
        
        # best feature and best alpha for the current step
        best_alpha = None
        best_index = None
        
        for i in L:
            x = X[:, i]
            alpha = _cal_alpha(X, x, best_x, r, d)
            if alpha is not None:
                if (best_alpha is None) or (best_alpha > alpha):
                    best_alpha = alpha
                    best_index = i
                
        best_x = X[:, best_index]
        beta = _cal_beta(best_alpha, beta, d)
        
        beta_path = np.column_stack((beta_path, beta))
#         beta_path.append(list(beta))
        
        r = y - X.dot(beta)
        
        # prepare for the next step
        e = np.zeros(m)
        np.put(e, best_index, 1)
        E = np.column_stack((E, e))
        
        d = _cal_d(X, m, E, r)
        
        A.append(best_index)
        L = list(set(F) - set(A))
        
    # 4. the last step 
    ## no next best feature, so best_alpha calculate method changes
    if L == []:
        #last alpha, such that last_x.T.dot(last_r) = 0
        last_alpha = _last_alpha(X, r, x, d)
        last_beta = _cal_beta(last_alpha, beta, d)
        beta = last_beta
        
        beta_path = np.column_stack((beta_path, beta))
#         beta_path.append(list(beta))
        
    # A is also the feature chosen order
    return beta, beta_path, A

In [4]:
# Example: Using sklearn diabetes dataset
from sklearn.datasets import load_diabetes

data, target = load_diabetes(return_X_y=True)

In [5]:
beta, beta_path, A = lars(data, target)
print('selected features ordered as select order are', A)
print('coefs', beta)

selected features ordered as select order are [2, 8, 3, 6, 1, 9, 4, 7, 5, 0]
coefs [ -0.47623169 -11.40703082  24.72625713  15.42967916 -37.68035801
  22.67648701   4.80620008   8.422084    35.73471316   3.21661161]


In [6]:
# Compare to sklearn's Lars to check if it is correct
from sklearn.linear_model import Lars

# using StandardScaler instead
standScaler = StandardScaler()
scaled_data = standScaler.fit_transform(data, target)
sklearn_lars = Lars(normalize=False)
sklearn_lars.fit(scaled_data, target)

print('sklearn version lars selected features(ordered) are', sklearn_lars.active_)
print('sklearn version lars coefs', sklearn_lars.coef_)

print(np.allclose(sklearn_lars.coef_, beta))
print(np.allclose(sklearn_lars.active_, A))
print(np.allclose(beta_path, sklearn_lars.coef_path_))

sklearn version lars selected features(ordered) are [2, 8, 3, 6, 1, 9, 4, 7, 5, 0]
sklearn version lars coefs [ -0.47623169 -11.40703082  24.72625713  15.42967916 -37.68035801
  22.67648701   4.80620008   8.422084    35.73471316   3.21661161]
True
True
True
